In [1]:
import os
import urllib.request
import pandas as pd
import csv
import re
import time
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from functools import wraps
import errno
import signal

In [2]:
class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.setitimer(signal.ITIMER_REAL,seconds) #used timer instead of alarm
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result
        return wraps(func)(wrapper)
    return decorator

In [3]:
# 크롬 시작 && 네이버 오픈
def openNaver(driver):
    driver.get("https://www.naver.com/")

In [4]:
# 로그인 버튼 클릭
def clickLoginBt(driver):
    naver_login = '''//*[@id="account"]/a'''
    driver.find_element_by_xpath(naver_login).click()

In [5]:
# 네이버 로그인
def loginNaver(driver):
    user_id = 'wjdtnwjdtn55'
    user_password = 'zmfhffldgkwk%6'

    # time.sleep(1)

    # 유저 아이디 비번 복붙으로 넣어주기
    tag_id = driver.find_element_by_name('id') 
    tag_pw = driver.find_element_by_name('pw')


    # 아이디 입력
    tag_id.click()
    pyperclip.copy(user_id)
#     print(user_id)
    # 맥은 커맨드 윈도우는 컨트롤
    tag_id.send_keys(Keys.COMMAND, "v")


    # 비번 입력
    tag_pw.click()
    pyperclip.copy(user_password)
#     print(user_password)
    # 맥은 커맨드 윈도우는 컨트롤
    tag_pw.send_keys(Keys.COMMAND, "v")


    # 로그인 버튼 클릭 (아이디 비번 입력 후에)
    login_btn = driver.find_element_by_id('log.login') 
    login_btn.click()

    # 로그인 실패시 처리
    try:
        login_error = driver.find_element_by_css_selector('#err_common > div > p') 
        print('로그인 실패 > ', login_error.text) 
    except: 
        print('로그인 성공')

In [6]:
def gotoStartPositiopn(driver, url):
#     # 중고나라 메인 페이지로 이동
#     notice_list_url_start = "https://cafe.naver.com/joonggonara"
#     test = "https://cafe.naver.com/joonggonara/864954962"
#     driver.get(notice_list_url_start)
    
#     # 중고나라 에어팟 게시판으로 이동
#     all_trade_docs_list = '''//*[@id="menuLink412"]'''
#     driver.find_element_by_xpath(all_trade_docs_list).click()
    
#     # frame switch
#     driver.switch_to.frame("cafe_main")
    
#     # 가장 최근 게시글 클릭 (시작 지점)
#     top_of_the_docs = '''//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a'''
#     driver.find_element_by_xpath(top_of_the_docs).click()
# #     driver.get(test)
# #     driver.switch_to.frame("cafe_main")

# #     무적 크롤러
    driver.get(url)
    driver.switch_to.frame("cafe_main")

## 목록을 타고 내려가면서 글을 바꿔줌

In [7]:
# @timeout(5)
def travel(nums, driver, url):
    aaa = driver.find_elements_by_class_name("tit_area")
    aa = driver.find_element_by_xpath('''//*[@id="app"]/div/div/div[1]/div[2]/a[2]''')
#     //*[@id="app"]/div/div/div[5]/div[1]/div/div[2]/div/ul/li[4]/div[1]

    aa.click()
#     try:
#         aaa[3].click()
#     except:
#         print("-", end = "")

In [8]:
def getAll(driver, dont_do_that):
    category = driver.find_element_by_xpath('''//*[@id="app"]/div/div/div[2]/div[1]/div[1]/a''').text   
        
    if category[0] == "M" or category in dont_do_that:
        list_all = []
        return 
    else:
#         상품의 상태 정보(판매, 판매 완료 등)와 상품명 
        product_name = driver.find_element_by_class_name("ProductName").text
        current_status = product_name[:2]
        product_name = product_name[3:]
        
#         게시글 작성자의 카페 내 정보
        nick_name = driver.find_element_by_class_name("nickname").text
        date = driver.find_element_by_class_name("date").text
        cafe_ranking = driver.find_element_by_class_name("nick_level").text
        
#         상품의 가격 정보
        price = driver.find_element_by_class_name("cost").text
        
#         상품의 거래 세부 내용 (결제 방식, 거래 방식)
        trade_detail = driver.find_elements_by_class_name("list_detail")
        trade_detail = [t.text for t in trade_detail]
        
#         list_detail이란 클래스 네임을 가진 elements가 여러개이기 때문에 쪼개고 그 중 원하는 것을 찾아가는 과정
        if trade_detail[0] in ["미개봉", "거의 새 것", "사용감 있음"]:
            payment = trade_detail[1]
            trade = trade_detail[2]
        else:
            payment = trade_detail[0]
            trade = trade_detail[1]
        
#         결제 방법에서 불필요한 내용 제거 -> 직접 결제 혹은 안전 결제의 형태로 추출
        payment = payment.split("\n")
        payment = payment[0]
        
#         본인인증 여부 확인
        try:
            can_trust = driver.find_element_by_class_name("info_good").text
        except:
            can_trust = "본인인증을 하지 않은 판매자입니다."
            
#         본문 내용 크롤링
        try:
            text = driver.find_element_by_class_name("se-component-content").text
            if len(text) == 0:
                try:
                    text = driver.find_elements_by_class_name("se-main-container")
                    text = [t.text for t in text]
                except:
                    text = "본문 없음"
        except:
            text = driver.find_elements_by_class_name("ContentRenderer")
            text = [t.text for t in text]
                
#         판매자 연락처 정보 확인 거래 완료된 상품은 불가능
#         time.sleep(0.3)
#         if current_status == "판매":
#             driver.find_element_by_class_name("btn_text").click()
#             time.sleep(0.4)
#             contact_info = driver.find_element_by_class_name("contact_box").text
# #             contact_info = [t.text for t in contact_info]
#             contact_info = contact_info.split("\n")
#             if len(contact_info) > 2:
#                 contact_info = contact_info[0:2]
#         else:
#             contact_info = ["거래 완료로 연락처 확인 불가"]
            
#         이미지 정보 가져오기
        try:
            image = driver.find_elements_by_css_selector('img.se-image-resource')
            if len(image) != 0:
                imgs = [link.rect for link in image]
                imgs_length = len(imgs)
            else:
                imgs = []
                imgs_length = 0 
        except:
            try:
                image = driver.find_elements_by_class_name("article_img ATTACH_IMAGE")
                if len(image) != 0:
                    imgs = [link.rect for link in image]
                    imgs_length = len(imgs)
                else:
                    imgs = []
                    imgs_length = 0
            except:
                imgs = []
                imgs_length = 0
                
    list_all = [category, nick_name, cafe_ranking, date, current_status, product_name, price, payment, trade, can_trust, text, imgs, imgs_length]
    return list_all

In [9]:
def goToCsv(path, docs, colum):
    junggo = pd.DataFrame(docs)
    try:
        junggo.columns = colum
    except:
        print("N", end = "")
        return
    
    if not os.path.exists(path):
        junggo.to_csv(path, index = False, mode = "w", encoding = "utf-8-sig")
    else:
        junggo.to_csv(path, index = False, mode = "a", encoding = "utf-8-sig", header = False)    

In [10]:
def getDriver():
#     options = webdriver.ChromeOptions()
#     options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

    driver = webdriver.Chrome(executable_path="/Users/jeongxoo/Desktop/web crolling/chromedriver")
    driver.implicitly_wait(3)
    return driver

In [11]:
def whatDoIDo(driver, path, colum, cnt, restart, length, url):
    doc = []
#     크롤링 대상이 아닌 게시판 목록 모음
    dont_do_that = ["자유게시판","부동산 매물","앱 가입인사","공지사항","이용정책","중고나라 뉴스","중나 매거진",
                    "카페 1:1 고객센터","중고나라 광고 문의하기","잡았다 요놈!","사기예방법","실시간 사기 제보","중나 이용팁",
                    "슬기로운 중나생활정보","제품 무료나눔","먹을거리 무료나눔","손글씨/이미지나눔","중고거래 후기",
                    "불량거래 후기","어려운 이웃 사연","회원 사진 올리기","컴퓨터 자유게시판","물물교환","자유게시판",
                    "이벤트","당첨자발표/종료","중나 할인특구","꿀딜특구","할인특가","재고 급처분","몰래하는 공구",
                    "중고폰 슈퍼핫딜","앵글선반 초특가","동부회원권거래소 부산","인터넷가입 인증협력사","최저가골프/콘도회원권", 
                    "라식,라섹 게시판","골프/콘도회원권 급매","푸드마켓", "파트너 중고폰 사고팔기", "파트너 중고폰 안전결제"]
    
    if cnt == length:
        restart = True
    
    for i in range(1, 600):
        cnt += 1
        
        travel(i, driver, url)       
        doc.clear()
        time.sleep(0.2)
        try:
            doc.append(getAll(driver, dont_do_that))
        except:
            doc.append([])

            
        time.sleep(0.2)
        goToCsv(path = path, docs = doc, colum = colum)
        
        
        if i % 50 == 0:
            print(i)

    driver.find_element_by_xpath('''//*[@id="app"]/div/div/div[2]/div[1]/div[3]/a[2]''').click()
    url = pyperclip.paste()
    print("Restart")
    driver.quit()
    return url

In [12]:
def main(path, colum, count):
    restart = False
    cnt = 0
    url = "https://cafe.naver.com/joonggonara/865531167"
    
    while restart != True:
        driver = getDriver()
        # 네이버 이동
        openNaver(driver)

        # 로그인 실행
        clickLoginBt(driver)
        loginNaver(driver)
         # 시작 지점으로 이동
        gotoStartPositiopn(driver, url)

        # 크롤링
        url = whatDoIDo(driver, path, colum, cnt, restart, count, url)

In [ ]:
path = "JUNGNA_AIRPOD_TRADE_0914.csv"
colum = ["category", "nick_name", "cafe_ranking", "date", "current_status", "product_name", "price", "payment", "trade", "can_trust", "text", "imgs", "imgs_length"]
main(path, colum, 30000)